In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({"A":["AA","BB","CC","AA","AA","EE"],
                   "B":["XYC","SSS","SSS","AAG","AFA","FFD"],
                   "C":["GXYC","SGSS","SGSS","AHAG","SFGS","HAFA"],
                  })

In [3]:
df.A = df.A.astype("category")
df.B = df.B.astype("category")
df.C = df.C.astype("category")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   A       6 non-null      category
 1   B       6 non-null      category
 2   C       6 non-null      category
dtypes: category(3)
memory usage: 738.0 bytes


## Label Encoding

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
cat_features = ["A", "B", "C"]
df_le = pd.DataFrame()
#1
l_encoded = df[cat_features].apply(encoder.fit_transform)
#2
for feature in cat_features:
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(df[feature])
    df_le[feature+"_encoded"] = encoded

In [6]:
df_le

,A_encoded,B_encoded,C_encoded
0,0,4,1
1,1,3,4
2,2,3,4
3,0,0,0
4,0,1,3
5,3,2,2


In [7]:
l_encoded

,A,B,C
0,0,4,1
1,1,3,4
2,2,3,4
3,0,0,0
4,0,1,3
5,3,2,2


## One-hot Encoding

In [8]:
df = pd.DataFrame({"Bolge":["A","A","B","C","D","B","E","C" ], "Sayi":[12,21,44,45,43,76,34,12]})

In [9]:
df

,Bolge,Sayi
0,A,12
1,A,21
2,B,44
3,C,45
4,D,43
5,B,76
6,E,34
7,C,12


In [10]:
#pandas get_dummies()
dummy_encoded = pd.get_dummies(df["Bolge"], prefix="Bolge")
dummy_encoded

,Bolge_A,Bolge_B,Bolge_C,Bolge_D,Bolge_E
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
5,0,1,0,0,0
6,0,0,0,0,1
7,0,0,1,0,0


In [11]:
df.join(dummy_encoded)

,Bolge,Sayi,Bolge_A,Bolge_B,Bolge_C,Bolge_D,Bolge_E
0,A,12,1,0,0,0,0
1,A,21,1,0,0,0,0
2,B,44,0,1,0,0,0
3,C,45,0,0,1,0,0
4,D,43,0,0,0,1,0
5,B,76,0,1,0,0,0
6,E,34,0,0,0,0,1
7,C,12,0,0,1,0,0


In [12]:
# Scikit-Learn OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
X = df.Bolge.values
X = X.reshape(-1,1)
oh_encoded = encoder.fit_transform(X)
ohe = pd.DataFrame(oh_encoded.toarray(), columns=["oh_A", "oh_B", "oh_C", "oh_D", "oh_E"], dtype="int32")
ohe

,oh_A,oh_B,oh_C,oh_D,oh_E
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
5,0,1,0,0,0
6,0,0,0,0,1
7,0,0,1,0,0


In [13]:
df.join(ohe)

,Bolge,Sayi,oh_A,oh_B,oh_C,oh_D,oh_E
0,A,12,1,0,0,0,0
1,A,21,1,0,0,0,0
2,B,44,0,1,0,0,0
3,C,45,0,0,1,0,0
4,D,43,0,0,0,1,0
5,B,76,0,1,0,0,0
6,E,34,0,0,0,0,1
7,C,12,0,0,1,0,0


## Count Encoding
Count encoding replaces each categorical value with the number of times it appears in the dataset. 

In [14]:
import category_encoders as ce
count_enc = ce.CountEncoder(
count_encoded = count_enc.fit_transform(df["Bolge"])
df.join(count_encoded.add_suffix("_count"))

SyntaxError: invalid syntax (<ipython-input-14-d42fd366bd52>, line 4)

### Count Encoding
cat_features = ['ip', 'app', 'device', 'os', 'channel']
train, valid, test = get_data_splits(clicks)
### Create the count encoder
count_enc = ce.CountEncoder(cols=cat_features)

### Learn encoding from the training set
count_enc.fit(train[cat_features])

### Apply encoding to the train and validation sets as new columns
### Make sure to add `_count` as a suffix to the new columns
train_encoded = train.join(count_enc.transform(train[cat_features]).add_suffix('_count')) <br>
valid_encoded = valid.join(count_enc.transform(valid[cat_features]).add_suffix('_count'))

### Target Encoding
Target encoding replaces a categorical value with the average value of the target for that value of the feature

### Create the target encoder. You can find this easily by using tab completion.
### Start typing ce. the press Tab to bring up a list of classes and functions.
target_enc = ce.TargetEncoder(cols=cat_features)

### Learn encoding from the training set. Use the 'is_attributed' column as the target.
target_enc.fit(train[cat_features], train['is_attributed'])

### Apply encoding to the train and validation sets as new columns
### Make sure to add `_target` as a suffix to the new columns
train_encoded = train.join(target_enc.transform(train[cat_features]).add_suffix('_target')) <br>
valid_encoded = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))


### CatBoost Encoding
This is similar to target encoding in that it's based on the target probablity for a given value. However with CatBoost, for each row, the target probability is calculated only from the rows before it. The CatBoost encoder is supposed to work well with the LightGBM model.

### Remove IP from the encoded features
cat_features = ['app', 'device', 'os', 'channel'] <br>
train, valid, test = get_data_splits(clicks)
### Create the CatBoost encoder
cb_enc = ce.CatBoostEncoder(cols=cat_features, random_state=7)

### Learn encoding from the training set
cb_enc.fit(train[cat_features], train['is_attributed'])

### Apply encoding to the train and validation sets as new columns
### Make sure to add `_cb` as a suffix to the new columns
train_encoded = train.join(cb_enc.transform(train[cat_features]).add_suffix('_cb')) <br>
valid_encoded = valid.join(cb_enc.transform(valid[cat_features]).add_suffix('_cb'))


## Feature Generation
Creating new features from the raw data is one of the best ways to improve your model. 

## Feature Selection
Often you'll have hundreds or thousands of features after various encodings and feature generation. This can lead to two problems. First, the more features you have, the more likely you are to overfit to the training and validation sets. This will cause your model to perform worse at generalizing to new data.

Secondly, the more features you have, the longer it will take to train your model and optimize hyperparameters. Also, when building user-facing products, you'll want to make inference as fast as possible. Using fewer features can speed up inference at the cost of predictive performance.

To help with these issues, you'll want to use feature selection techniques to keep the most informative features for your model.


### Univariate Feature Selection

With SelectKBest, we define the number of features to keep, based on the score from the scoring function. Using .fit_transform(features, target) we get back an array with only the selected features.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

### L1 regularization

Univariate methods consider only one feature at a time when making a selection decision. Instead, we can make our selection using all of the features by including them in a linear model with L1 regularization. This type of regularization (sometimes called Lasso) penalizes the absolute magnitude of the coefficients, as compared to L2 (Ridge) regression which penalizes the square of the coefficients.

As the strength of regularization is increased, features which are less important for predicting the target are set to 0. This allows us to perform feature selection by adjusting the regularization parameter. We choose the parameter by finding the best performance on a hold-out set, or decide ahead of time how many features to keep.

For regression problems you can use sklearn.linear_model.Lasso, or sklearn.linear_model.LogisticRegression for classification. These can be used along with sklearn.feature_selection.SelectFromModel to select the non-zero coefficients. Otherwise, the code is similar to the univariate tests.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

train, valid, _ = get_data_splits(baseline_data)

X, y = train[train.columns.drop("outcome")], train['outcome']

# Set the regularization parameter C=1
logistic = LogisticRegression(C=1, penalty="l1", solver='liblinear', random_state=7).fit(X, y)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X)
X_new

# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X.index,
                                 columns=X.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]
